# Task 4 - Proxy Target Variable Engineering 

In [1]:
import numpy as np
from datasets import Dataset, ClassLabel, Sequence
import pandas as pd
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForTokenClassification
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append(os.path.abspath("../src"))
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("..")
print(os.getcwd()) 

d:\KAIM 5,6,7\Week-5\Credit-Risk-Probability-Model


In [ ]:
from proxy_variable import calculate_rfm, cluster_rfm, assign_high_risk_label, label_high_risk_customers
# 1. Load your original transaction data
transactions = pd.read_csv("data.csv")

# 2. Process data to get model features (use the process_data function from Task 3)
processed = process_data(transactions)

# 3. Create proxy target variable
final_df = label_high_risk_customers(transactions, processed)

# 4. Save to CSV or use for model training
final_df.to_csv("model_ready_data.csv", index=False)

# Show result
print(final_df[['CustomerId', 'is_high_risk']].value_counts())